In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
%config IPCompleter.greedy=True

In [ ]:
# File to Load (Remember to Change These)
school_data_to_load = ('schools_complete.csv')
student_data_to_load = ('students_complete.csv')

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
alldf = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [ ]:
alldf = alldf.rename(columns = {"student_name": "Student", "gender": "Gender", "grade": "Grade", "school_name": "School", "reading_score": "Reading Score", "math_score": "Math Score", "type": "Type", "size": "Size", "budget": "Budget"})

In [ ]:
# district summary
count_passing_math = alldf[alldf["Math Score"] >= 70].count()["School"]
count_passing_reading = alldf[alldf["Reading Score"] >= 70].count()["School"]

avg_score = (alldf["Math Score"].mean() + alldf["Reading Score"].mean()) / 2
total_schools = alldf['School'].nunique()
total_students = alldf['Student ID'].count()
total_budget = school_data['budget'].sum()
avg_math = alldf['Math Score'].mean()
avg_reading = alldf['Reading Score'].mean()
percent_passing_math = (count_passing_math / (alldf['Student ID'].count())) * 100
percent_passing_reading = (count_passing_reading / (alldf['Student ID'].count())) * 100
percent_passing_both = (percent_passing_math + percent_passing_reading)/ 2

In [ ]:
District_Summary = pd.DataFrame({
    "Total Schools": total_schools,
    "Total Students": total_students,
    "Total Budget": total_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_reading,
    "% Passing Math": [percent_passing_math],
    "% Passing Reading": [percent_passing_reading],
    "% Overall Passing Rate": [avg_score]}
)

In [ ]:
District_Summary

In [ ]:
# school summary
school_types = school_data.set_index(["school_name"])["type"]
per_school_counts = alldf["School"].value_counts()
per_school_budget = alldf.groupby(["School"]).mean()["Budget"]
per_student_budget = per_school_budget/ per_school_counts

avg_math_score = alldf.groupby(["School"]).mean()["Math Score"]
avg_reading_score = alldf.groupby(["School"]).mean()["Reading Score"]

school_passing_math =  alldf[alldf["Math Score"] >= 70].groupby("School").count()["Student"]
school_passing_reading =  alldf[alldf["Reading Score"] >= 70].groupby("School").count()["Student"]

In [ ]:
percent_passing_math = school_passing_math / per_school_counts * 100
percent_passing_reading = school_passing_reading / per_school_counts * 100
overall_passing_rate = (percent_passing_math + percent_passing_reading) / 2

In [ ]:
School_Summary = pd.DataFrame({
    "School Type":school_types, 
    "Total Students":per_school_counts,
    "Total School Budget":per_school_budget, 
    "Per Student Budget":per_student_budget,
    "Average Math Score":avg_math_score, 
    "Average Reading Score":avg_reading_score,
    "% Passing Math":percent_passing_math,
    "% Passing Reading":percent_passing_reading,
    "% Overall Passing Rate":overall_passing_rate
})            

In [ ]:
School_Summary = School_Summary[["School Type","Total Students","Total School Budget","Per Student Budget",
                                "Average Math Score","Average Reading Score","% Passing Math","% Passing Reading",
                                 "% Overall Passing Rate"]]

In [ ]:
School_Summary["Total School Budget"] = School_Summary["Total School Budget"].map("${:,.2f}".format)
School_Summary["Per Student Budget"] = School_Summary["Per Student Budget"].map("${:,.2f}".format)

In [ ]:
School_Summary

In [ ]:
#top schools
top_schools = School_Summary.sort_values(["% Overall Passing Rate"], ascending = False)
top_schools


In [ ]:
#bottom schools
bottom_schools = School_Summary.sort_values(["% Overall Passing Rate"], ascending = True)
bottom_schools


In [ ]:
# math score by grade
nineth =  alldf[alldf["Grade"] == "9th"].groupby("School").mean()["Math Score"]
tenth =  alldf[alldf["Grade"] == "10th"].groupby("School").mean()["Math Score"]
eleventh =  alldf[alldf["Grade"] == "11th"].groupby("School").mean()["Math Score"]
twelveth =  alldf[alldf["Grade"] == "12th"].groupby("School").mean()["Math Score"]


In [ ]:
math_score_by_grade_df = pd.DataFrame({
    "9th":nineth,
    "10th":tenth,
    "11th":eleventh,
    "12th":twelveth
})            

In [ ]:
Math_Score_by_Grade = math_score_by_grade_df[["9th", "10th", "11th", "12th"]]
Math_Score_by_Grade.index.name = None

In [ ]:
Math_Score_by_Grade

In [ ]:
# reading score by grade
nineth_grade_score =  alldf[alldf["Grade"] == "9th"].groupby("School").mean()["Reading Score"]
tenth_grade_score =  alldf[alldf["Grade"] == "10th"].groupby("School").mean()["Reading Score"]
eleventh_grade_score =  alldf[alldf["Grade"] == "11th"].groupby("School").mean()["Reading Score"]
twelveth_grade_score =  alldf[alldf["Grade"] == "12th"].groupby("School").mean()["Reading Score"]

In [ ]:
reading_score_by_grade_df = pd.DataFrame({
    "9th":nineth_grade_score, 
    "10th":tenth_grade_score,
    "11th":eleventh_grade_score,
    "12th":twelveth_grade_score
})            

In [ ]:
Reading_Score_by_Grade = reading_score_by_grade_df[["9th", "10th", "11th", "12th"]]
Reading_Score_by_Grade.index.name = None

In [ ]:
Reading_Score_by_Grade

In [ ]:
# scores by school spending
spending_bins = [0, 585, 610, 635, 99999]
spending_ranges = ["<585", "585-610", "610-635", "635<"]

In [ ]:
School_Summary["Spending Ranges (Per Student)"] = pd.cut(per_student_budget, spending_bins, labels = spending_ranges)

In [ ]:
spending_math_score = School_Summary.groupby(["Spending Ranges (Per Student)"]).mean()['Average Math Score']
spending_reading_score = School_Summary.groupby(["Spending Ranges (Per Student)"]).mean()['Average Reading Score']
spending_passing_math =  School_Summary.groupby(["Spending Ranges (Per Student)"]).mean()['% Passing Math']
spending_passing_reading =  School_Summary.groupby(["Spending Ranges (Per Student)"]).mean()['% Passing Reading']
overall_passing_rate =  (spending_math_score + spending_reading_score) / 2

In [ ]:
Spending_Score = pd.DataFrame({"Average Math Score":spending_math_score, "Average Reading Score":spending_reading_score,
                               "% Passing Math":spending_passing_math,"% Passing Reading":spending_passing_reading,
                                    "Overall Passing Rate":overall_passing_rate}) 

In [ ]:
Spending_Score = Spending_Score[["Average Math Score", "Average Reading Score",
                               "% Passing Math","% Passing Reading","Overall Passing Rate"]]

In [ ]:
Spending_Score

In [ ]:
# scores by school size
size_bins = [0, 1000, 2000, 5000]
size_ranges = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
School_Summary["School Size"] = pd.cut(School_Summary["Total Students"], size_bins, labels = size_ranges)

In [ ]:
avg_math_score = School_Summary.groupby(["School Size"]).mean()['Average Math Score']
avg_reading_score = School_Summary.groupby(["School Size"]).mean()['Average Reading Score']
percent_passing_math =  School_Summary.groupby(["School Size"]).mean()['% Passing Math']
percent_passing_reading =  School_Summary.groupby(["School Size"]).mean()['% Passing Reading']
overall_passing_rate = School_Summary.groupby(["School Size"]).mean()['% Overall Passing Rate']

In [ ]:
Size_Score = pd.DataFrame({"Average Math Score":avg_math_score, "Average Reading Score":avg_reading_score,
                               "% Passing Math":percent_passing_math,"% Passing Reading":percent_passing_reading,
                                    "% Overall Passing Rate":overall_passing_rate})

In [ ]:
Size_Score = Size_Score[["Average Math Score", "Average Reading Score",
                               "% Passing Math","% Passing Reading","% Overall Passing Rate"]]

In [ ]:
Size_Score

In [ ]:
# scores by school type 
avg_math_score = School_Summary.groupby(["School Type"]).mean()['Average Math Score']
avg_reading_score = School_Summary.groupby(["School Type"]).mean()['Average Reading Score']
percent_passing_math =  School_Summary.groupby(["School Type"]).mean()['% Passing Math']
percent_passing_reading =  School_Summary.groupby(["School Type"]).mean()['% Passing Reading']
overall_passing_rate = School_Summary.groupby(["School Type"]).mean()['% Overall Passing Rate']

In [ ]:
Type_Score = pd.DataFrame({"Average Math Score":avg_math_score, "Average Reading Score":avg_reading_score,
                               "% Passing Math":percent_passing_math,"% Passing Reading":percent_passing_reading,
                                    "% Overall Passing Rate":overall_passing_rate})            

In [ ]:
Type_Score = Type_Score[["Average Math Score", "Average Reading Score",
                               "% Passing Math","% Passing Reading","% Overall Passing Rate"]]

In [ ]:
Type_Score